# Red Tide Detection in South Florida

Detecting Red Tide events in southwest Florida based on known events from 25 October 2024.


## What is red tide?
Red tide is a type harmful algal bloom (HAB). In southern FLorida, these HAMs are typically observed by *K. brevis*, an algae which is known to produce potent neurotoxins that affect both marine life and humans. 

In [1]:
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# ee.Authenticate()
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
# Applies scaling factors.
def apply_scale_factors(image):
    """Apply scaling factors to Landsat9 surface relfectance data"""
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

def cloudMask(image):
    clouds = ee.Algorithms.Landsat.simpleCloudScore(image).select(['cloud'])
    return image.updateMask(clouds.lt(10))

def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B5', 'B4']))

In [4]:
# Create a basemap
Map = geemap.Map(center=[26.4, -81.99], zoom=7, basemap='TERRAIN')

In [5]:
# Get area of interest around South Florida
roi = ee.Geometry.Rectangle([-83.4277, 29.5, -78.7351, 24.5])

In [17]:
# Load a region in which to compute the mean and display it.
counties = ee.FeatureCollection('TIGER/2016/Counties')
manatee = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Manatee')).first())
Map.addLayer(ee.Image().paint(manatee, 0, 2), 
             {'palette': 'yellow'}, 'Manatee')

## Load Datasets

### Surface reflectance

In [11]:
# Access the Landsat 9 Level 2, Collection 2, Tier 1 dataset
landsat_surf_reflectance = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") \
    .filterDate('2024-10-15', '2024-10-26') \
    .filterBounds(roi)

landsat_surf_reflectance = landsat_surf_reflectance.map(apply_scale_factors)

vis_landsat_rgb = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}
Map.add_layer(landsat_surf_reflectance.clip(roi), vis_landsat_rgb, 'True Color (Landsat9)')

AttributeError: 'ImageCollection' object has no attribute 'clip'

### Sea surface temperature

In [8]:
# Access the GCOM-C/SGLI L3 Sea Surface Temperature (V3) dataset
sst_jaxa = ee.ImageCollection("JAXA/GCOM-C/L3/OCEAN/SST/V3") \
    .filterDate('2024-10-15', '2024-10-26') \
    .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D')) \
    .filterBounds(roi) \
    .select('SST_AVE') 

# Visualization parameters
vis_params_sst =  {
    'min': 20,
    'max': 35,  # Typical SST range in Celsius; adjust based on your region
    'palette': ['3500a8','0800ba','003fd6',
                '00aca9','77f800','ff8800',
                'b30000','920000','880000']
}

# Convert to Celcius
mean_sst = sst_jaxa.mean().multiply(0.0012).add(-10);

Map.addLayer(mean_sst.clip(roi), vis_params_sst, 'Mean SST (GCOM-C)')

### Chlorophyll-a data

Some datasets, like the GCOM-C/SGLI L3 Chlorophyll-a Concentration (V3), calculate the Chlorophyll-a concentration. Others, like the Sentinel-3 OLCI EFR, require calculation from the images

In [9]:
#  GCOM-C/SGLI L3 Chlorophyll-a Concentration (V3) dataset for chlor-a measurements 
jaxa_chlor_a = ee.ImageCollection("JAXA/GCOM-C/L3/OCEAN/CHLA/V3") \
    .filterBounds(roi) \
    .filterDate('2024-10-15', '2024-10-26') \
    .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))

# Get the mean SST image over the selected time range
mean_chlor_a = jaxa_chlor_a.select('CHLA_AVE').mean().multiply(0.0016).log10();

visParams_jaxa_chlor_a = {
    'min': -2,
    'max': 2,
    'palette': ['3500a8','0800ba','003fd6',
                '00aca9','77f800','ff8800',
                'b30000','920000','880000']
}

Map.addLayer(mean_chlor_a.clip(roi), visParams_jaxa_chlor_a, '[Chlor. A] (JAXA)')

In [45]:
# Sentinel3 dataset to extract ocean color for chlor-a measurements
SENTINEL3 = ee.ImageCollection("COPERNICUS/S3/OLCI") \
    .filterDate('2024-10-15', '2024-10-26') \
    .filterBounds(roi) 


SENTINEL3_chlora = SENTINEL3.select("Oa05_radiance").median().multiply(0.0100953)

visParams_SENTINEL3_chlora = {
      'min': 0,
      'max': 4,
      'gamma': 1.5,
    }

# SENTINEL3_rgb = SENTINEL3.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance']) \
#           .median() \
#         .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])) # Convert to radiance units.


# visParams_SENTINEL3_rgb = {
#       'min': 0,
#       'max': 4,
#       'gamma': 2,
#     }

# Map.addLayer(SENTINEL3_rgb.clip(roi), visParams_SENTINEL3_rgb, 'SENTINEL3_rgb');

In [43]:
SENTINEL3_chlora = SENTINEL3.select("Oa05_radiance").median().multiply(0.0100953)
visParams_SENTINEL3_chlora = {
      'min': 0,
      'max': 4,
      'gamma': 1.5,
    }
# Map.addLayer(SENTINEL3_chlora, visParams_SENTINEL3_chlora, 'SENTINEL3_chlora');

In [36]:
SENTINEL3_chlora.getInfo()

{'type': 'Image',
 'bands': [{'id': 'Oa05_radiance',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [37]:
# Base imagery dataset
SENTINEL2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(roi) \
    .filterDate('2024-10-15', '2024-10-26') \
    # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    # .limit(10)

meanSENTINEL2 = SENTINEL2.mean()
vizParamsSENTINEL2 = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 5000, 'gamma':2}
Map.addLayer(meanSENTINEL2.clip(roi), vizParamsSENTINEL2, 'Mean SENTINEL2 Image')

In [40]:
# VIIRS = ee.ImageCollection("NASA/VIIRS/002/VNP09GA") \
#              .filterDate('2024-10-17', '2024-10-31') \
#              # .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))

In [10]:
Map

Map(center=[26.4, -81.99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [16]:
# Sort the collection by date in descending order and get the last n images
n = 5  # Specify the number of most recent images you want
last_n_images = SENTINEL2.sort('system:time_start', False).limit(n)

# Print the dates of the last n images (optional)
dates = last_n_images.aggregate_array('system:time_start') \
    .map(lambda date: ee.Date(date).format('YYYY-MM-dd')).getInfo()
print("Dates of the last n images:", dates)

Dates of the last n images: ['2024-10-25', '2024-10-25', '2024-10-25', '2024-10-25', '2024-10-25']
